In [1]:
library(tidyverse)

source("../../evaluation_utils/plots_eda.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: viridisLite



# Load data

In [2]:
norm_vs_HGSC = c("GSE6008", "GSE26712", "GSE40595", "GSE69428", "GSE38666", "GSE14407")

Preprocess the data by loading the data from the CSV files and displaying the first few rows of the data.

Save the data plots to the `plots` directory for each sub-datasets.

In [3]:
all_expression <- NULL
all_metadata <- NULL

i <- 1
for(dataset in norm_vs_HGSC){
    metadata <- read.table(paste0("before/", dataset, "/intermediate/", i, "_metadata_short.tsv"), header = TRUE, sep = "\t")
    expr_data <- read.table(paste0("before/", dataset, "/intermediate/expr_GBacc_MaxVar.tsv"), header = TRUE, sep = "\t") 
    print(paste0("Processing dataset: ", dataset, "platform: ", metadata$platform_id[1]))
    metadata <- metadata %>% mutate(Dataset = dataset) %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage2, Grade2, ClinicopathologicSubtypes, Dataset) %>%
        filter(Status == "normal" | Status == "ovarian tumour" & HistSubtypes == "high-grade serous carcinoma") %>%
        # outliers
        filter(!(Sample_geo_ID %in% c("GSM1701038", "GSM898305", "GSM997639")))

    # for plots
    plt_meta <- metadata #%>%
        # filter((Status == "ovarian tumour" & HistSubtypes == "high-grade serous carcinoma" | 
                # Status == "normal"))

    plt_expression <- expr_data %>% column_to_rownames("row_ID")
    plt_meta <- plt_meta[plt_meta$Sample_geo_ID %in% colnames(expr_data),]
    plt_expression <- expr_data[,plt_meta$Sample_geo_ID]

    # plots
    print("..plotting..")
    #  plot
    plot_res <- plots_normtumor(plt_expression[sample(1:nrow(plt_expression), 10000),], plt_meta, dataset, ', norm_vs_tumor')
    layout <- (plot_res[[1]] + plot_res[[2]] ) / 
            (plot_res[[3]] )
    ggsave(paste0("before/", dataset, "/intermediate/norm_vs_tumor_plot.png"), 
                plot = layout, width = 12, height = 12)

    if(is.null(all_metadata)){
        all_metadata <- metadata
        all_expression <- expr_data
    } else {
        
        all_metadata <- rbind(all_metadata, metadata)

        # shared features analysis:
        # all_expression <- inner_join(all_expression, expr_data, by = "row_ID")

        # all features analysis (union):
        all_expression <- full_join(all_expression, expr_data, by = "row_ID")
    }
}

[1] "Processing dataset: GSE6008platform: GPL96"
[1] "..plotting.."


Warning message:
“`aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.”
No id variables; using all as measure variables



[1] "Processing dataset: GSE26712platform: GPL96"
[1] "..plotting.."


No id variables; using all as measure variables



[1] "Processing dataset: GSE40595platform: GPL570"
[1] "..plotting.."


...

No id variables; using all as measure variables



[1] "Processing dataset: GSE38666platform: GPL570"
[1] "..plotting.."


No id variables; using all as measure variables



[1] "Processing dataset: GSE14407platform: GPL570"
[1] "..plotting.."


No id variables; using all as measure variables



In [4]:
all_metadata %>%
    group_by(Dataset, Status) %>% summarise(n())

`summarise()` has grouped output by 'Dataset'. You can override using the
`.groups` argument.


Dataset,Status,n()
<chr>,<chr>,<int>
GSE14407,normal,12
GSE14407,ovarian tumour,12
GSE26712,normal,10
GSE26712,ovarian tumour,185
GSE38666,normal,12
GSE38666,ovarian tumour,18
GSE40595,normal,6
GSE40595,ovarian tumour,31
GSE6008,normal,4


In [5]:
all_metadata %>%
    group_by(Dataset, Status, HistSubtypes) %>% summarise(n())

`summarise()` has grouped output by 'Dataset', 'Status'. You can override using
the `.groups` argument.


Dataset,Status,HistSubtypes,n()
<chr>,<chr>,<chr>,<int>
GSE14407,normal,NA,12
GSE14407,ovarian tumour,high-grade serous carcinoma,12
GSE26712,normal,NA,10
GSE26712,ovarian tumour,high-grade serous carcinoma,185
GSE38666,normal,NA,12
GSE38666,ovarian tumour,high-grade serous carcinoma,18
GSE40595,normal,NA,6
GSE40595,ovarian tumour,high-grade serous carcinoma,31
GSE6008,normal,NA,4


In [6]:
all_metadata <- all_metadata %>%
    mutate(Status = as.factor(Status),
             Stage2 = as.factor(Stage2),
             Grade2 = as.factor(Grade2),
             ClinicopathologicSubtypes = as.factor(ClinicopathologicSubtypes),
             Dataset = as.factor(Dataset))

all_expression <- all_expression %>% column_to_rownames("row_ID")
all_metadata <- all_metadata[all_metadata$Sample_geo_ID %in% colnames(all_expression),]
all_expression <- all_expression[,all_metadata$Sample_geo_ID]

print(paste0("Number of samples in type_1_early_vs_late: ", ncol(all_expression)))
print(paste0("Number of genes in type_1_early_vs_late: ", nrow(all_expression)))

[1] "Number of samples in type_1_early_vs_late: 332"
[1] "Number of genes in type_1_early_vs_late: 51276"


In [7]:
#  plot
plot_res <- plots_normtumor(all_expression[sample(1:nrow(all_expression), 10000),], 
                            all_metadata, "", 'norm_vs_tumor')
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
         (plot_res[[3]] )
# ggsave("before/plots/norm_vs_HGSC_plot.png", 
#              plot = layout, width = 14, height = 12)
ggsave("before/plots/norm_vs_tumor_plot_UNION.png", 
             plot = layout, width = 14, height = 12)



No id variables; using all as measure variables

Warning message:
“Removed 1304028 rows containing non-finite outside the scale range
(`stat_density()`).”
Warning message:
“Removed 1304028 rows containing non-finite outside the scale range
(`stat_boxplot()`).”
Warning message:
“Removed 1304028 rows containing non-finite outside the scale range
(`stat_summary()`).”


# Central limma:removeBatchEffect correction

In [8]:
# remove batch effect from GSE68928
design <- model.matrix(~all_metadata$Status)
colnames(design) <- c("Intercept", "HGSC")
corrected_expr <- limma::removeBatchEffect(all_expression, batch = all_metadata$Dataset, design = design) %>% as.data.frame()

# save corrected data to after
write.table(corrected_expr %>% rownames_to_column("Gene"), "after/central_corrected_UNION.tsv", sep = "\t", quote = FALSE, row.names = FALSE)


# # plot plots for norm_vs_tumor
plot_res <- plots_normtumor(
    # take random 10000 rows 
    corrected_expr[sample(1:nrow(corrected_expr), 10000),], all_metadata, " ", "norm_vs_tumor")
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
    (plot_res[[3]] )
ggsave("before/plots/norm_vs_tumor_plot_Rcorrected_UNION.png", 
             plot = layout, width = 14, height = 12)


Warning message:
“Partial NA coefficients for 30148 probe(s)”
No id variables; using all as measure variables

Warning message:
“Removed 1303140 rows containing non-finite outside the scale range
(`stat_density()`).”
Warning message:
“Removed 1303140 rows containing non-finite outside the scale range
(`stat_boxplot()`).”
Warning message:
“Removed 1303140 rows containing non-finite outside the scale range
(`stat_summary()`).”


In [9]:
# save expression and metadata in folder
# all in one file
write.table(all_metadata, "before/all_metadata.tsv", sep = "\t", quote = FALSE, row.names = FALSE)
# write.table(all_expression %>% rownames_to_column("Gene"), "before/all_expression.tsv", sep = "\t", quote = FALSE, row.names = FALSE)
write.table(all_expression %>% rownames_to_column("Gene"), "before/all_expression_UNION.tsv", sep = "\t", quote = FALSE, row.names = FALSE)


In [10]:
path_to_after_data <- "after"

design <- design %>% as.data.frame()
rownames(design) <- all_metadata$Sample_geo_ID

for(center in unique(all_metadata$Dataset)){
    print(center)
    path_to_before <- paste0("before/", center, "/")

    design_center = design[all_metadata[all_metadata$Dataset == center,]$Sample_geo_ID,]['HGSC']

    # if(!dir.exists(path_to_after)){
    #     dir.create(path_to_after)
    # }

    write.table(design_center %>% rownames_to_column('file'),
        file = paste0(path_to_before, "design.tsv"), sep = "\t", quote = F, row.names = F, col.names = T)

    # take also the expression data
    expr_center <- all_expression[,all_metadata$Sample_geo_ID[all_metadata$Dataset == center]]
    print(dim(expr_center))
    write.table(expr_center %>% rownames_to_column("Gene"), file = paste0(path_to_before, "expr_for_correction_UNION.tsv"), sep = "\t", quote = F, row.names = F, col.names = T)
}

[1] "GSE6008"
[1] 51276    27
[1] "GSE26712"
[1] 51276   195
[1] "GSE40595"
[1] 51276    37
[1] "GSE69428"
[1] 51276    19
[1] "GSE38666"
[1] 51276    30
[1] "GSE14407"
[1] 51276    24


# Session information


In [11]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /home/yuliya/miniforge3/envs/fedRBE/lib/libopenblasp-r0.3.28.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] viridis_0.6.5     viridisLite_0.4.2 ggsci_3.2.0       umap_0.2.10.0    
 [5] patchwork_1.3.0   gridExtra_2.3     lubridate_1.9.3   forcats_1.0.0    
 [9] stringr_1.5.1     dplyr_1.1.4       purrr_1.0.2    